In [53]:
# Entity-Specific Billing System - Clean Version
# This program generates bills with entity-specific logos and metadata

# Install required packages if not available
import sys
import subprocess

def install_package(package):
    """Install a package if it's not already installed"""
    try:
        __import__(package)
    except ImportError:
        print(f"Installing {package}...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])

# Install required packages
required_packages = [
    "pandas>=2.1",
    "matplotlib>=3.5", 
    "Pillow>=9.0",
    "numpy>=1.20",
    "ipywidgets>=8.0",
    "reportlab>=4.0",
    "PyPDF2>=3.0",
    "openpyxl>=3.0"
]

for package in required_packages:
    install_package(package)

# Now import the packages
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from datetime import datetime
import os
from PIL import Image
import numpy as np
import ipywidgets as widgets
from IPython.display import display, HTML

# Entity Configuration
ENTITY_CONFIG = {
    "ANESTHESIOLOGY & PERIOPERATIVE MEDICINE SPECIALISTS": {
        "name": "Anesthesiology & Perioperative Medicine Specialists",
        "address": "9531 S. Santa Monica Blvd, Suite #1413",
        "city_state_zip": "Beverly Hills, CA 90210-4503",
        "tax_id": "Tax I.D. #99-1043743",
        "phone": "Billing Office: (424) 388-0042",
        "email": "apmsmedicalbilling@gmail.com",
        "logo_path": "../Material/apms_logo.jpg",
        "billing_team": "APMS Billing Team",
        "Service Type": "Anesthesia Services"
    },
    "UNIVERSITY SURGICAL INSTITUTE, LLC": {
        "name": "UNIVERSITY SURGICAL INSTITUTE, LLC",
        "address": "8200 Stockdale HWY, Suite M-10287",
        "city_state_zip": "Bakersfield, CA 93311",
        "tax_id": "Tax I.D. #93-2514800",
        "phone": "Billing Office: (424) 388-0042",
        "email": "billing@universitysurgicalinstitute.com",
        "logo_path": "../Material/usi_logo.png",
        "billing_team": "University Surgical Institute Billing Team",
        "Service Type": "Surgical Facility Services"
    },
    "SCOPES HEALTH INC.": {
        "name": "SCOPES Health, Inc.",
        "address": "9531 S. Santa Monica Blvd, Suite #1413",
        "city_state_zip": "Beverly Hills, CA 90210-4503",
        "tax_id": "Tax I.D. #99-1006225",
        "phone": "Billing Office: (424) 388-0042",
        "email": "billing@scopeshealthcare.com",
        "logo_path": "../Material/scopes_logo.jpg",
        "billing_team": "SCOPES Health Billing Team",
        "Service Type": "Professional Medical Services"
    }
}


Installing pandas>=2.1...
Installing matplotlib>=3.5...
Installing Pillow>=9.0...
Installing numpy>=1.20...
Installing ipywidgets>=8.0...
Installing reportlab>=4.0...
Installing PyPDF2>=3.0...
Installing openpyxl>=3.0...


In [54]:
# Enable Jupyter widgets extension
try:
    import subprocess
    import sys
    
    # Try to enable the widget extension
    try:
        subprocess.check_call([sys.executable, "-m", "jupyter", "nbextension", "enable", "--py", "widgetsnbextension", "--sys-prefix"], 
                            capture_output=True)
        print("✅ Widget extension enabled successfully")
    except subprocess.CalledProcessError as e:
        print("⚠️  Could not enable widget extension automatically. Widgets may still work.")
        print("If widgets don't work, try restarting your Jupyter kernel.")
    except FileNotFoundError:
        print("⚠️  Jupyter command not found. Widgets may still work.")
        
except Exception as e:
    print(f"⚠️  Error enabling widgets: {e}")
    print("Widgets may still work without explicit enabling.")


⚠️  Error enabling widgets: Popen.__init__() got an unexpected keyword argument 'capture_output'
Widgets may still work without explicit enabling.


In [55]:
# Load main datasets
CombinedCases = pd.read_excel("../DataBase/CombinedCasesFinal.xlsx")
PI_Attorney = pd.read_csv("../DataBase/PI Attorney.csv")
PI_Patient = pd.read_csv("../DataBase/PI Patients.csv")
Case_Rep = pd.read_csv("../DataBase/Case Rep.csv")

print("Data loaded successfully!")
print(f"CombinedCases: {CombinedCases.shape}")
print(f"PI_Attorney: {PI_Attorney.shape}")
print(f"PI_Patient: {PI_Patient.shape}")
print(f"Case_Rep: {Case_Rep.shape}")

# Check the actual column names
print(f"\nCombinedCases columns: {list(CombinedCases.columns)}")
print(f"PI_Attorney columns: {list(PI_Attorney.columns)}")
print(f"PI_Patient columns: {list(PI_Patient.columns)}")
print(f"Case_Rep columns: {list(Case_Rep.columns)}")


Data loaded successfully!
CombinedCases: (497, 25)
PI_Attorney: (61, 11)
PI_Patient: (102, 15)
Case_Rep: (73, 4)

CombinedCases columns: ['First Name', 'Last Name', 'Service Type', 'Physcian Name', 'Date of Service', 'Location Name', 'Date Billed', 'Service Provided', 'Full Service Description', 'Law Firm', 'Attorney', 'Case Rep', 'Item Charge', 'Pmt Date', 'Paid Amount', 'Collection %', 'Lien Reduction %', 'Settlement Accepted', 'Discount Date', 'Discount Amount', 'Case Status', 'Charge Amount', 'Settlement %', 'Settlement Date', 'Entity']
PI_Attorney columns: ['Attorney Name', 'Firm', 'Address', 'City', 'State', 'Zip', 'Work Phone', 'Mobile Phone', 'Assistant Name', 'Email', 'Asst Phone']
PI_Patient columns: ['Id', 'Patient Fname', 'Patient Lname', 'DOB', 'Patient Phone', 'Address', 'City', 'State', 'Zip', 'Physcians', 'Case Reps', 'Attorneys', 'DOI', 'Patient Email', 'Patient ID']
Case_Rep columns: ['Case Rep Name', 'CR Email', 'CR Phone', 'CR Fax']


In [56]:
# Data processing and merging functions
TRACE_ENABLED = False

def set_trace(enabled: bool):
    global TRACE_ENABLED
    TRACE_ENABLED = bool(enabled)

_trace_events = []
_trace_store = {}

def trace(event: str, **kwargs):
    if TRACE_ENABLED:
        _trace_events.append({"event": event, **kwargs})


def normalize_text(s):
    """Lowercase, strip, collapse whitespace, safe if series contains NaN."""
    return s.fillna('').astype(str).str.strip().str.lower().str.replace(r'\s+', ' ', regex=True)

def clean_money(series):
    """Remove $ , and convert to float. Returns float series (NaN if not convertible)."""
    return pd.to_numeric(series.astype(str).str.replace(r'[\$,]', '', regex=True).str.replace('--','').str.strip(), errors='coerce')

def parse_dates(df, cols):
    """Inplace parse of multiple date columns; returns df."""
    for c in cols:
        if c in df.columns:
            df[c] = pd.to_datetime(df[c], errors='coerce', infer_datetime_format=True)
    return df

def _safe_to_datetime(val):
    """Parse dates robustly, handling MM/DD/YYYY, DD/MM/YYYY, and dash variants.
    Heuristic:
    - If the first token > 12 => dayfirst
    - If the second token > 12 => monthfirst
    - If both <= 12 => default to monthfirst (US style) unless dayfirst=True is explicitly requested
    """
    try:
        # Fast path for pandas Timestamp or datetime
        if hasattr(val, 'to_pydatetime') or hasattr(val, 'year'):
            return pd.to_datetime(val, errors='coerce')
        s = str(val).strip()
        if not s or s.lower() in {"nan", "none", "nat"}:
            return pd.NaT
        # Normalize separators
        sep = "/" if "/" in s else ("-" if "-" in s else None)
        if sep:
            parts = s.split(sep)
            if len(parts) == 3:
                try:
                    p0 = int(parts[0])
                    p1 = int(parts[1])
                except Exception:
                    # Fallback to general parser
                    return pd.to_datetime(s, errors='coerce', dayfirst=False)
                # Decide dayfirst
                if p0 > 12 and p1 <= 12:
                    return pd.to_datetime(s, errors='coerce', dayfirst=True)
                if p1 > 12 and p0 <= 12:
                    return pd.to_datetime(s, errors='coerce', dayfirst=False)
                # Ambiguous (both <= 12): try month-first then day-first if needed
                dt = pd.to_datetime(s, errors='coerce', dayfirst=False)
                if pd.isna(dt):
                    dt = pd.to_datetime(s, errors='coerce', dayfirst=True)
                return dt
        # Fallback generic
        return pd.to_datetime(s, errors='coerce', dayfirst=False)
    except Exception:
        return pd.NaT


def combined_parse_dates(df, cols):
    """Inplace parse of multiple date columns; returns df."""
    for c in cols:
        if c in df.columns:
            df[c] = df[c].apply(_safe_to_datetime)
    return df

def prepare_combined(df):
    df = df.copy()
    df['patient_full'] = normalize_text(df['First Name']) + ' ' + normalize_text(df['Last Name'])
    if 'Attorney' in df.columns:
        df['attorney_norm'] = normalize_text(df['Attorney'])
    combined_parse_dates(df, ['Date of Service', 'Date Billed', 'Pmt Date', 'Discount Date', 'Settlement Date'])
    for col in ['Item Charge', 'Paid Amount', 'Charge Amount', 'Settlement Accepted', 'Discount Amount', 'Collection %', 'Lien Reduction %', 'Settlement %']:
        if col in df.columns:
            df[col] = clean_money(df[col])
    if 'Date Billed' in df.columns:
        df['billing_period'] = df['Date Billed'].dt.to_period('M').astype(str).fillna('unbilled')
    else:
        df['billing_period'] = 'unbilled'
    return df

def prepare_patients(df):
    df = df.copy()
    df['patient_full'] = normalize_text(df['Patient Fname']) + ' ' + normalize_text(df['Patient Lname'])
    parse_dates(df, ['DOB', 'DOI'])
    if 'Patient Phone' in df.columns:
        df['Patient Phone'] = df['Patient Phone'].astype(str).replace('nan','',regex=False)
        # Format phone numbers to (XXX) XXX-XXXX format
        def format_phone(phone_str):
            if pd.isna(phone_str) or phone_str == '' or phone_str == 'nan':
                return 'Unknown'
            # Remove all non-digit characters
            digits = ''.join(filter(str.isdigit, str(phone_str)))
            if len(digits) == 10:
                return f"({digits[:3]}) {digits[3:6]}-{digits[6:]}"
            elif len(digits) == 11 and digits[0] == '1':
                return f"({digits[1:4]}) {digits[4:7]}-{digits[7:]}"
            else:
                return str(phone_str)  # Return original if can't format
        df['Patient Phone'] = df['Patient Phone'].apply(format_phone)
    # Ensure ZIP is a clean string (no .0)
    if 'Zip' in df.columns:
        def clean_zip(z):
            if pd.isna(z):
                return ''
            s = str(z).strip()
            # If numeric like 93307.0 -> 93307
            try:
                f = float(s)
                if f.is_integer():
                    return str(int(f))
            except Exception:
                pass
            # Remove trailing .0 if present
            if s.endswith('.0'):
                s = s[:-2]
            return s
        df['Zip'] = df['Zip'].apply(clean_zip)
    return df

def prepare_attorneys(df):
    df = df.copy()
    df['attorney_norm'] = normalize_text(df['Attorney Name'])
    if 'Zip' in df.columns:
        df['Zip'] = df['Zip'].apply(lambda z: str(int(z)) if (pd.notna(z) and float(z).is_integer()) else (str(z) if pd.notna(z) else ''))
    return df

def prepare_case_reps(df):
    df = df.copy()
    df['case_rep_norm'] = normalize_text(df['Case Rep Name'])
    return df

def merge_all(combined, patients, attorneys, case_reps):
    """Returns merged dataframe with patient, attorney, and case rep fields attached."""
    c = prepare_combined(combined)
    p = prepare_patients(patients)
    a = prepare_attorneys(attorneys)
    cr = prepare_case_reps(case_reps)

    patient_cols = {}
    for col in ['DOB', 'DOI', 'Patient Phone', 'Address', 'City', 'State', 'Zip', 'Patient Email']:
        if col in p.columns:
            if col == 'Address':
                patient_cols[col] = 'patient_street'
            else:
                patient_cols[col] = f'patient_{col.lower().replace(" ", "_")}'
    
    # Create a combined address field
    if 'Address' in p.columns and 'City' in p.columns and 'State' in p.columns and 'Zip' in p.columns:
        # Create address for each row individually
        def create_address(row):
            address = str(row['Address']) if pd.notna(row['Address']) else ''
            city = str(row['City']) if pd.notna(row['City']) else ''
            state = str(row['State']) if pd.notna(row['State']) else ''
            zip_code = str(row['Zip']) if pd.notna(row['Zip']) else ''
            
            # Clean up the values
            address = address.replace('nan', '').replace('None', '').strip()
            city = city.replace('nan', '').replace('None', '').strip()
            state = state.replace('nan', '').replace('None', '').strip()
            zip_code = zip_code.replace('nan', '').replace('None', '').strip()
            
            # Combine parts
            parts = [part for part in [address, city, state, zip_code] if part]
            return ', '.join(parts) if parts else 'Unknown Address'
        
        p['patient_address'] = p.apply(create_address, axis=1).astype(str)
        patient_cols['patient_address'] = 'patient_address'
    p_rename = p[['patient_full'] + list(patient_cols.keys())].rename(columns=patient_cols)

    merged = c.merge(p_rename, on='patient_full', how='left')

    att_pick = []
    att_map = {}
    for col in ['Attorney Name', 'Firm', 'Address', 'City', 'State', 'Zip', 'Work Phone', 'Email']:
        if col in a.columns:
            att_pick.append(col)
            att_map[col] = 'attorney_' + col.lower().replace(' ', '_')

    a_small = a[['attorney_norm'] + att_pick].rename(columns=att_map)
    merged = merged.merge(a_small, on='attorney_norm', how='left')
    
    # Add Case Rep information
    if 'Case Rep' in merged.columns:
        merged['case_rep_norm'] = normalize_text(merged['Case Rep'])
        cr_pick = []
        cr_map = {}
        for col in ['Case Rep Name', 'CR Email', 'CR Phone', 'CR Fax']:
            if col in cr.columns:
                cr_pick.append(col)
                cr_map[col] = 'case_rep_' + col.lower().replace(' ', '_').replace('cr_', '')
        
        cr_small = cr[['case_rep_norm'] + cr_pick].rename(columns=cr_map)
        merged = merged.merge(cr_small, on='case_rep_norm', how='left')
    
    trace('merge_all_complete', merged_shape=tuple(merged.shape))
    return merged


In [57]:
# Merge all data
merged = merge_all(CombinedCases, PI_Patient, PI_Attorney, Case_Rep)
print(f"Merged data: {merged.shape}")

print(f"Data loaded successfully!")
print(f"Found {len(merged)} records")
print(f"Entities: {list(merged['Entity'].dropna().unique())}")


Merged data: (520, 50)
Data loaded successfully!
Found 520 records
Entities: ['SCOPES HEALTH INC.', 'UNIVERSITY SURGICAL INSTITUTE, LLC', 'ANESTHESIOLOGY & PERIOPERATIVE MEDICINE SPECIALISTS']


/var/folders/sb/pr7bfv397g12byqs3qhx90tc0000gp/T/ipykernel_3432/2711078932.py:28: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df[c] = pd.to_datetime(df[c], errors='coerce', infer_datetime_format=True)
/var/folders/sb/pr7bfv397g12byqs3qhx90tc0000gp/T/ipykernel_3432/2711078932.py:28: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df[c] = pd.to_datetime(df[c], errors='coerce', infer_datetime_format=True)


In [58]:
def get_patient_bill_data_open_only(merged_df, first_name, last_name, dob, entity=None):
    """Get comprehensive bill data for a specific patient with 'Open' case status only."""
    import pandas as pd
    
    first_name_norm = first_name.strip().lower()
    last_name_norm = last_name.strip().lower()
    
    if isinstance(dob, str):
        try:
            dob_parsed = pd.to_datetime(dob, format='%m/%d/%Y')
        except Exception:
            try:
                dob_parsed = pd.to_datetime(dob)
            except Exception:
                return None, None, None, None
    else:
        dob_parsed = dob
    
    mask = (
        (merged_df['First Name'].str.lower().str.strip() == first_name_norm) &
        (merged_df['Last Name'].str.lower().str.strip() == last_name_norm) &
        (pd.to_datetime(merged_df['patient_dob'], errors='coerce') == dob_parsed) &
        (merged_df['Case Status'] == 'Open')  # Only include 'Open' cases
    )
    
    if entity and str(entity).strip():
        mask &= (merged_df['Entity'].astype(str).str.strip().str.lower() == str(entity).strip().lower())
    
    patient_data = merged_df[mask]
    
    if patient_data.empty:
        print(f"No 'Open' case found with name: {first_name} {last_name}, DOB: {dob}, Entity: {entity or 'Any'}")
        return None, None, None, None
    
    # Use the most recent row by Date of Service to reflect latest Case Rep/attorney info
    try:
        patient_data_sorted = patient_data.sort_values(
            by=['Date of Service'],
            key=lambda s: s.apply(_safe_to_datetime)
        ).reset_index(drop=True)
        first_row = patient_data_sorted.iloc[-1]
    except Exception:
        # Fallback: last row in current order
        first_row = patient_data.iloc[-1]
    
    # Get patient address as string, not Series
    patient_address = first_row.get('patient_address', 'Unknown Address')
    if hasattr(patient_address, 'iloc'):
        # If it's a Series, get the first value
        patient_address = str(patient_address.iloc[0]) if len(patient_address) > 0 else 'Unknown Address'
    else:
        patient_address = str(patient_address)
    
    patient_info = {
        'Patient Fname': first_row.get('First Name', 'Unknown'),
        'Patient Lname': first_row.get('Last Name', 'Unknown'),
        'DOB': first_row.get('patient_dob', 'Unknown'),
        'DOI': first_row.get('patient_doi', 'Unknown'),
        'Patient Phone': first_row.get('patient_patient_phone', 'Unknown'),
        'patient_address': patient_address,
        'patient_street': first_row.get('patient_street', ''),
        'patient_city': first_row.get('patient_city', ''),
        'patient_state': first_row.get('patient_state', ''),
        'patient_zip': first_row.get('patient_zip', ''),
        'Entity': first_row.get('Entity', 'Unknown Entity')  # Add Entity information for proper branding
    }
    
    attorney_info = {
        'Attorney Name': first_row.get('attorney_attorney_name', first_row.get('Attorney', 'Unknown Attorney')),
        'Case Rep': first_row.get('Case Rep', 'Unknown'),
        'Firm': first_row.get('attorney_firm', first_row.get('Law Firm', 'Unknown Firm')),
        'Address': first_row.get('attorney_address', 'Unknown Address'),
        'City': first_row.get('attorney_city', 'Unknown City'),
        'State': first_row.get('attorney_state', 'Unknown State'),
        'Zip': first_row.get('attorney_zip', 'Unknown Zip'),
        'attorney_email': first_row.get('attorney_email', ''),
        'case_rep_email': first_row.get('case_rep_email', ''),
        'case_rep_phone': first_row.get('case_rep_phone', ''),
        'case_rep_fax': first_row.get('case_rep_fax', '')
    }
    
    procedures = []
    total_amount = 0.0
    
    for _, row in patient_data.iterrows():
        charge_amount = 0.0
        if pd.notna(row.get('Charge Amount')):
            charge_amount = float(row['Charge Amount'])
        elif pd.notna(row.get('Item Charge')):
            charge_amount = float(row['Item Charge'])
        
        procedure = {
            'Date of Service': row.get('Date of Service', 'Unknown'),
            'Full Service Description': row.get('Full Service Description', 'Unknown Service'),
            'Item Charge': row.get('Item Charge', 0),
            'Charge Amount': row.get('Charge Amount', 0),
            'Final Charge': charge_amount
        }
        
        procedures.append(procedure)
        total_amount += charge_amount
    
    # Sort procedures by Date of Service in chronological order
    def sort_key(procedure):
        date_of_service = procedure['Date of Service']
        if pd.isna(date_of_service) or date_of_service == 'Unknown':
            return pd.Timestamp.min  # Put unknown dates at the end
        try:
            if isinstance(date_of_service, str):
                return pd.to_datetime(date_of_service, errors='coerce')
            return pd.to_datetime(date_of_service, errors='coerce')
        except:
            return pd.Timestamp.min  # Put invalid dates at the end
    
    procedures.sort(key=sort_key)
    
    return patient_info, attorney_info, procedures, total_amount


In [59]:
def generate_pdf_bill_with_exact_w9_scaling(patient_info, attorney_info, procedures, total_amount, output_path="bill.pdf"):
    """Generate a PDF bill with entity-specific titles, service types, and W-9 forms scaled to exact page dimensions."""
    try:
        from reportlab.lib.pagesizes import letter
        from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer, Table, TableStyle, Image
        from reportlab.lib import colors
        from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
        from reportlab.lib.units import inch
        from PyPDF2 import PdfReader, PdfWriter
        from reportlab.pdfgen import canvas
        from reportlab.lib.utils import ImageReader
        import io
    except ImportError:
        print("Installing required packages...")
        import subprocess
        subprocess.check_call([sys.executable, "-m", "pip", "install", "reportlab", "PyPDF2"])
        from reportlab.lib.pagesizes import letter
        from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer, Table, TableStyle, Image
        from reportlab.lib import colors
        from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
        from reportlab.lib.units import inch
        from PyPDF2 import PdfReader, PdfWriter
        from reportlab.pdfgen import canvas
        from reportlab.lib.utils import ImageReader
        import io
    
    # First, create the bill content
    doc = SimpleDocTemplate(output_path, pagesize=letter, 
                            rightMargin=0.75*inch, leftMargin=0.75*inch,
                            topMargin=0.75*inch, bottomMargin=0.75*inch)
    styles = getSampleStyleSheet()
    elements = []
    
    # Header - Dynamic based on entity
    entity_name = patient_info.get('Entity', 'UNIVERSITY SURGICAL INSTITUTE, LLC')
    
    if entity_name in ENTITY_CONFIG:
        config = ENTITY_CONFIG[entity_name]
        header_text = f"""
        <b>{config['name']}</b><br/>
        {config['address']}<br/>
        {config['city_state_zip']}<br/><br/>
        {config['tax_id']}<br/>
        {config['phone']}<br/>
        {config['email']}<br/>
        """
        # Add logo if available
        logo_path = config.get('logo_path', '')
        if logo_path and os.path.exists(logo_path):
            try:
                # Load image to get natural dimensions
                from PIL import Image as PILImage
                pil_img = PILImage.open(logo_path)
                img_width, img_height = pil_img.size
                
                # Calculate natural aspect ratio and scale to reasonable size
                max_width = 2.5 * inch
                max_height = 1.5 * inch
                
                # Calculate scale to fit within max dimensions while maintaining aspect ratio
                scale_w = max_width / img_width
                scale_h = max_height / img_height
                scale = min(scale_w, scale_h)
                
                logo_width = img_width * scale
                logo_height = img_height * scale
                
                logo = Image(logo_path, width=logo_width, height=logo_height)
                logo.hAlign = 'RIGHT'
                
                header_table_data = [
                    [Paragraph(header_text, styles["Normal"]), logo]
                ]
                header_table = Table(
                                    header_table_data,
                                    colWidths=[3.5*inch, 2.5*inch],
                                    rowHeights=[1.25*inch]  # fixed header row height
                                    )
                header_table.setStyle(TableStyle([
                    ("ALIGN", (0,0), (0,0), "LEFT"),
                    ("ALIGN", (1,0), (1,0), "RIGHT"),
                    ("VALIGN", (0,0), (-1,-1), "TOP"),
                    ("LEFTPADDING", (0,0), (-1,-1), 0),
                    ("RIGHTPADDING", (0,0), (-1,-1), 0),
                    ("TOPPADDING", (0,0), (-1,-1), 0),
                    ("BOTTOMPADDING", (0,0), (-1,-1), 0),
                ]))
                elements.append(header_table)
            except Exception as e:
                print(f"Could not load logo: {e}")
                elements.append(Paragraph(header_text, styles["Normal"]))
        else:
            elements.append(Paragraph(header_text, styles["Normal"]))
    else:
        config = ENTITY_CONFIG.get('UNIVERSITY SURGICAL INSTITUTE, LLC', {})
        header_text = f"""
        <b>UNIVERSITY SURGICAL INSTITUTE, LLC</b><br/>
        8200 Stockdale HWY, Suite M-10287<br/>
        Bakersfield, CA 93311<br/><br/>
        Tax I.D. #93-2514800<br/>
        Billing Office: +1 424 388 0042<br/>
        billing@universitysurgicalinstitute.com<br/>
        """
        elements.append(Paragraph(header_text, styles["Normal"]))
    
    elements.append(Spacer(1, 12))
    
    # Date and Delivery
    current_date = datetime.now().strftime("%m-%d-%Y")
    
    # Use Case Rep email if available, otherwise fall back to attorney email, then config email
    delivery_email = attorney_info.get('case_rep_email', '') or attorney_info.get('attorney_email', '') or config.get('email', 'billing@universitysurgicalinstitute.com')
    
    header_table_data = [
        [f"Date: {current_date}", ""],
        [f"Delivery via Email only: {delivery_email}", ""]
    ]
    
    header_table = Table(header_table_data, colWidths=[3*inch, 3*inch])
    header_table.setStyle(TableStyle([
        ("ALIGN", (0,0), (0,-1), "LEFT"),
        ("ALIGN", (1,0), (1,-1), "RIGHT"),
        ("FONTSIZE", (0,0), (-1,-1), 10),
        ("FONTNAME", (0,0), (-1,-1), "Helvetica"),
    ]))
    
    elements.append(header_table)
    elements.append(Spacer(1, 20))
    
    # Billed To and Patient sections
    billed_to_data = [
        [Paragraph("<b>Billed To</b>", styles['Normal'])],
        [Paragraph(attorney_info.get('Attorney Name', 'Unknown Attorney'), styles['Normal'])],
        [Paragraph(f"Attn: {attorney_info.get('Case Rep', 'Unknown')}", styles['Normal'])],
        [Paragraph(attorney_info.get('Firm', 'Unknown Firm'), styles['Normal'])],
        [Paragraph(attorney_info.get('Address', 'Unknown Address'), styles['Normal'])],
        [Paragraph(f"{attorney_info.get('City', 'Unknown')}, {attorney_info.get('State', 'Unknown')} {attorney_info.get('Zip', 'Unknown')}", styles['Normal'])]
    ]
    
    patient_dob_str = patient_info.get('DOB', 'Unknown')
    if hasattr(patient_dob_str, 'strftime'):
        patient_dob_str = patient_dob_str.strftime('%m/%d/%Y')
    else:
        patient_dob_str = str(patient_dob_str)
    
    patient_doi_str = patient_info.get('DOI', 'Unknown')
    if hasattr(patient_doi_str, 'strftime'):
        patient_doi_str = patient_doi_str.strftime('%m/%d/%Y')
    else:
        patient_doi_str = str(patient_doi_str)
    
    # Build full address in two lines
    street_line = patient_info.get('patient_street', '') or patient_info.get('patient_address', '') or 'Unknown Address'
    city = patient_info.get('patient_city', '')
    state = patient_info.get('patient_state', '')
    zip_code = patient_info.get('patient_zip', '')
    city_state_zip = ', '.join(filter(None, [city, state]))
    if zip_code:
        city_state_zip = f"{city_state_zip} {zip_code}" if city_state_zip else zip_code
    if not city_state_zip:
        city_state_zip = ''

    patient_data = [
        [Paragraph("<b>Patient</b>", styles['Normal'])],
        [Paragraph(f"{patient_info.get('Patient Fname', 'Unknown')} {patient_info.get('Patient Lname', 'Unknown')}", styles['Normal'])],
        [Paragraph(f"DOB: {patient_dob_str}", styles['Normal'])],
        [Paragraph(f"DOL: {patient_doi_str}", styles['Normal'])],
        [Paragraph(f"Address: {street_line}", styles['Normal'])],
        [Paragraph(f"{city_state_zip}", styles['Normal'])],
        [Paragraph(f"Tel: {patient_info.get('Patient Phone', 'Unknown')}", styles['Normal'])]
    ]
    
    combined_data = []
    max_rows = max(len(billed_to_data), len(patient_data))
    for i in range(max_rows):
        left = billed_to_data[i][0] if i < len(billed_to_data) else Paragraph("", styles['Normal'])
        right = patient_data[i][0] if i < len(patient_data) else Paragraph("", styles['Normal'])
        combined_data.append([left, right])
    
    info_table = Table(combined_data, colWidths=[3*inch, 3*inch])
    info_table.setStyle(TableStyle([
        ("ALIGN", (0,0), (-1,-1), "LEFT"),
        ("FONTSIZE", (0,0), (-1,-1), 10),
        ("FONTNAME", (0,0), (-1,-1), "Helvetica"),
        ("VALIGN", (0,0), (-1,-1), "TOP"),
        ("LEFTPADDING", (0,0), (-1,-1), 0),
        ("RIGHTPADDING", (0,0), (-1,-1), 6),
        ("TOPPADDING", (0,0), (-1,-1), 2),
        ("BOTTOMPADDING", (0,0), (-1,-1), 2),
    ]))
    
    elements.append(info_table)
    elements.append(Spacer(1, 20))
    
    # Title - Entity-specific based on service type
    title_style = ParagraphStyle(
        'TitleStyle',
        parent=styles['Heading2'],
        fontSize=14,
        fontName='Helvetica-Bold',
        alignment=1,
        leading=16
    )
    
    # Get entity-specific title
    if entity_name in ENTITY_CONFIG:
        service_type = ENTITY_CONFIG[entity_name].get('Service Type', 'Medical Services')
        if service_type == "Professional Medical Services":
            title_text = "BILLING STATEMENT & PROFESSIONAL CLAIM"
        elif service_type == "Surgical Facility Services":
            title_text = "BILLING STATEMENT & SURGICAL SERVICES"
        elif service_type == "Anesthesia Services":
            title_text = "BILLING STATEMENT & ANESTHESIA SERVICES"
        else:
            title_text = "BILLING STATEMENT & MEDICAL SERVICES"
    else:
        title_text = "BILLING STATEMENT & SURGICAL SERVICES"
    
    elements.append(Paragraph(f"<b>{title_text}</b>", title_style))
    elements.append(Spacer(1, 12))
    
    # Procedures Table
    body_style = ParagraphStyle('Body', parent=styles['Normal'], fontName='Helvetica', fontSize=10, leading=12)
    header_bold_style = ParagraphStyle('HeaderBold', parent=styles['Normal'], fontName='Helvetica-Bold', fontSize=10)
    
    table_data = [
        [Paragraph("DATE OF SERVICE", header_bold_style), Paragraph("DESCRIPTION", header_bold_style), Paragraph("CHARGES", header_bold_style)]
    ]
    
    for procedure in procedures:
        date_str = procedure.get('Date of Service', 'Unknown')
        if hasattr(date_str, 'strftime'):
            date_str = date_str.strftime('%m/%d/%Y')
        else:
            date_str = str(date_str)
        
        charge_amount = procedure.get('Final Charge', 0)
        charge_str = f"${charge_amount:,.2f}"
        desc_para = Paragraph(str(procedure.get('Full Service Description', 'Unknown Service')), body_style)
        table_data.append([Paragraph(date_str, body_style), desc_para, Paragraph(charge_str, body_style)])
    
    total_label = Paragraph("<b>Total Balance Due:</b>", body_style)
    total_value = Paragraph(f"<b>${total_amount:,.2f}</b>", body_style)
    table_data.append([Paragraph("", body_style), total_label, total_value])
    
    table = Table(table_data, colWidths=[1.2*inch, 4*inch, 1.2*inch])
    table.setStyle(TableStyle([
        ("BACKGROUND", (0,0), (-1,0), colors.grey),
        ("TEXTCOLOR", (0,0), (-1,0), colors.whitesmoke),
        ("ALIGN", (0,1), (0,-1), "CENTER"),
        ("ALIGN", (1,1), (1,-1), "LEFT"),
        ("ALIGN", (2,1), (2,-1), "RIGHT"),
        ("GRID", (0,0), (-1,-1), 0.5, colors.black),
        ("FONTNAME", (0,0), (-1,0), "Helvetica-Bold"),
        ("FONTSIZE", (0,0), (-1,-1), 10),
        ("VALIGN", (0,0), (-1,-1), "TOP"),
        ("LEFTPADDING", (0,0), (-1,-1), 6),
        ("RIGHTPADDING", (0,0), (-1,-1), 6),
        ("TOPPADDING", (0,0), (-1,-1), 6),
        ("BOTTOMPADDING", (0,0), (-1,-1), 6),
    ]))
    
    elements.append(table)
    elements.append(Spacer(1, 20))
    
    # Footer with entity-specific messages using configuration
    footer_style = ParagraphStyle(
        'FooterStyle',
        parent=styles['Normal'],
        fontSize=10,
        fontName='Helvetica',
        leading=12,
        alignment=0,
    )
    
    # Get service type and company name from configuration
    if entity_name in ENTITY_CONFIG:
        service_type = ENTITY_CONFIG[entity_name].get('Service Type', 'Medical Services')
        company_name = ENTITY_CONFIG[entity_name].get('name', 'Medical Services Provider')
        billing_team = ENTITY_CONFIG[entity_name].get('billing_team', 'Billing Team')
    else:
        service_type = "Medical Services"
        company_name = config.get('name', 'University Surgical Institute')
        billing_team = config.get('billing_team', 'University Surgical Institute Billing Team')
    
    footer_text = f"""
    <i>Thank you for choosing {company_name} for {service_type}. 
    Please remit payment for this claim to the mailing address and unique Tax I.D. number written above, 
    and as written on the attached W-9 form for this entity.</i>
    <br/><br/>
    Sincerely,<br/>
    {billing_team}
    """
    elements.append(Paragraph(footer_text, footer_style))
    
    # Build the bill PDF first
    doc.build(elements)
    
    # Now merge with W-9 form if it exists - with exact page scaling
    print(f"Looking for W-9 form for entity: {entity_name}")
    
    # Try multiple possible W-9 file names for each entity
    w9_paths = []
    
    # Generate possible W-9 file names for the entity
    entity_clean = entity_name.replace(' ', '_').replace(',', '').replace('.', '').replace('&', 'and')
    w9_paths.append(f"../Material/{entity_clean}_W9.pdf")
    w9_paths.append(f"../Material/{entity_clean}_W-9.pdf")
    w9_paths.append(f"../Material/{entity_clean}_w9.pdf")
    w9_paths.append(f"../Material/{entity_clean}_w-9.pdf")
    
    # Also try with different entity name formats
    if "ANESTHESIOLOGY" in entity_name.upper():
        w9_paths.extend([
            "../Material/APMS_W9.pdf",
            "../Material/APMS_W-9.pdf",
            "../Material/ANESTHESIOLOGY_W9.pdf",
            "../Material/ANESTHESIOLOGY_W-9.pdf",
            "../Material/ANESTHESIOLOGY_&_PERIOPERATIVE_MEDICINE_SPECIALISTS_W9.pdf"
        ])
    elif "UNIVERSITY" in entity_name.upper():
        w9_paths.extend([
            "../Material/USI_W9.pdf",
            "../Material/USI_W-9.pdf",
            "../Material/UNIVERSITY_W9.pdf",
            "../Material/UNIVERSITY_W-9.pdf"
        ])
    elif "SCOPES" in entity_name.upper():
        w9_paths.extend([
            "../Material/SCOPES_W9.pdf",
            "../Material/SCOPES_W-9.pdf",
            "../Material/SCOPES_HEALTH_W9.pdf",
            "../Material/SCOPES_HEALTH_W-9.pdf"
        ])
    
    # Debug: List all files in Material directory
    material_dir = "../Material"
    if os.path.exists(material_dir):
        print(f"Files in {material_dir}:")
        for file in os.listdir(material_dir):
            if file.lower().endswith('.pdf'):
                print(f"  - {file}")
    
    w9_found = False
    for w9_path in w9_paths:
        print(f"Checking: {w9_path}")
        if os.path.exists(w9_path):
            w9_found = True
            print(f"✅ Found W-9 form: {w9_path}")
            try:
                # Read the bill PDF
                bill_reader = PdfReader(output_path)
                w9_reader = PdfReader(w9_path)
                
                # Create a new PDF writer
                writer = PdfWriter()
                
                # Add all pages from the bill
                for page in bill_reader.pages:
                    writer.add_page(page)
                
                # Add W-9 pages with exact page scaling to match billing page dimensions
                for page in w9_reader.pages:
                    # Get the page dimensions
                    page_width = float(page.mediabox.width)
                    page_height = float(page.mediabox.height)

                    # Slightly crop edges to remove phone borders (zoom-in effect)
                    crop_margin = 24  # points (~0.33 inch)
                    try:
                        # Adjust mediabox to crop
                        new_llx = page.mediabox.left + crop_margin
                        new_lly = page.mediabox.bottom + crop_margin
                        new_urx = page.mediabox.right - crop_margin
                        new_ury = page.mediabox.top - crop_margin
                        # Ensure valid box
                        if (new_urx - new_llx) > 100 and (new_ury - new_lly) > 100:
                            page.mediabox.lower_left = (new_llx, new_lly)
                            page.mediabox.upper_right = (new_urx, new_ury)
                            # Update vars after crop
                            page_width = float(page.mediabox.width)
                            page_height = float(page.mediabox.height)
                    except Exception as _:
                        pass
                    
                    # Get the target dimensions (Letter size)
                    target_width = 612  # Letter width in points
                    target_height = 792  # Letter height in points
                    
                    # Calculate scaling factors
                    scale_x = target_width / page_width
                    scale_y = target_height / page_height
                    
                    # Use the smaller scale to ensure the page fits; add a tiny extra zoom
                    scale = min(scale_x, scale_y) * 1.02
                    
                    print(f"Original/trimmed W-9 dimensions: {page_width} x {page_height}")
                    print(f"Target dimensions: {target_width} x {target_height}")
                    print(f"Scaling factor (with zoom): {scale}")
                    
                    # Scale the page to match billing page dimensions
                    page.scale_by(scale)
                    writer.add_page(page)
                
                # Write the merged PDF
                with open(output_path, 'wb') as output_file:
                    writer.write(output_file)
                
                print(f"✅ W-9 form merged successfully with exact scaling: {w9_path}")
                break
                
            except Exception as e:
                print(f"❌ Could not merge W-9 form: {e}")
                print(f"Bill generated without W-9 form: {output_path}")
                break
    
    if not w9_found:
        print(f"❌ W-9 form not found for entity: {entity_name}")
        print(f"Searched paths:")
        for path in w9_paths:
            print(f"  - {path}")
        print(f"Bill generated without W-9 form: {output_path}")
    
    print(f"PDF bill generated successfully: {output_path}")
    return output_path


In [60]:
def create_patient_bill_open_only_exact_w9(merged_df, first_name, last_name, dob, output_path=None, entity=None):
    """Complete function to get patient data and generate a PDF bill for 'Open' cases only with exact W-9 scaling."""
    
    patient_info, attorney_info, procedures, total_amount = get_patient_bill_data_open_only(
        merged_df, first_name, last_name, dob, entity=entity
    )
    
    if patient_info is None:
        return None
    
    if output_path is None:
        safe_name = f"{first_name}_{last_name}_bill.pdf".replace(" ", "_")
        output_path = safe_name
    
    # If tracing enabled, write debug artifacts
    try:
        if TRACE_ENABLED:
            debug_dir = os.path.join(os.path.dirname(output_path), 'Debug')
            os.makedirs(debug_dir, exist_ok=True)
            import json
            with open(os.path.join(debug_dir, 'trace_events.json'), 'w', encoding='utf-8') as f:
                json.dump(_trace_events, f, indent=2, default=str)
    except Exception as _:
        pass
    
    pdf_path = generate_pdf_bill_with_exact_w9_scaling(patient_info, attorney_info, procedures, total_amount, output_path)
    
    return {
        'patient_info': patient_info,
        'attorney_info': attorney_info, 
        'procedures': procedures,
        'total_amount': total_amount,
        'pdf_path': pdf_path,
        'num_procedures': len(procedures)
    }


In [61]:
# Updated GUI Interface with Copy/Paste Support and Auto-Generated Save Location
# Input widgets with proper copy/paste support
first_name_w = widgets.Text(
    description='First Name', 
    placeholder='e.g., Michelle', 
    layout=widgets.Layout(width='200px'),
    style={'description_width': 'initial'}
)
last_name_w = widgets.Text(
    description='Last Name', 
    placeholder='e.g., Taylor', 
    layout=widgets.Layout(width='200px'),
    style={'description_width': 'initial'}
)
dob_w = widgets.Text(
    description='DOB', 
    placeholder='MM/DD/YYYY', 
    layout=widgets.Layout(width='200px'),
    style={'description_width': 'initial'}
)

# Entity dropdown
if 'Entity' in merged.columns:
    entity_values = sorted(list({str(v) for v in merged['Entity'].dropna().unique()}))
    entity_options = ['Any'] + entity_values
else:
    entity_options = ['Any']
entity_w = widgets.Dropdown(options=entity_options, value='Any', description='Entity')

generate_btn = widgets.Button(description='Generate Bill with W-9 (Open Cases Only)', button_style='primary', icon='file')
status_out = widgets.Output()
pdf_out = widgets.Output()

def on_generate_clicked(_):
    status_out.clear_output()
    pdf_out.clear_output()
    
    with status_out:
        fn = first_name_w.value.strip()
        ln = last_name_w.value.strip()
        dob_str = dob_w.value.strip()
        entity_sel = None if entity_w.value == 'Any' else entity_w.value

        if not fn or not ln or not dob_str:
            print('Please fill First Name, Last Name, and DOB.')
            return
        
        try:
            dob = pd.to_datetime(dob_str, format='%m/%d/%Y')
        except Exception:
            try:
                dob = pd.to_datetime(dob_str)
            except Exception:
                print('DOB format invalid. Use MM/DD/YYYY.')
                return
        
        # Create output directory structure: ../Output_Folder/{PatientFirstName}_{PatientLastName}_Bills
        output_base_dir = "../Output_Folder"
        invoice_name = f"{fn}_{ln}_Bills"
        output_dir = os.path.join(output_base_dir, invoice_name)
        os.makedirs(output_dir, exist_ok=True)
        
        # Get entity name for filename
        entity_name = entity_sel or 'Any'
        entity_clean = entity_name.replace(' ', '_').replace(',', '').replace('.', '').replace('&', 'and').replace('LLC', '').replace('INC', '').strip('_')
        
        pdf_name = f"{fn}_{ln}_Bill_{entity_clean}_{datetime.now().strftime('%Y%m%d')}.pdf".replace(' ', '_')
        pdf_path = os.path.join(output_dir, pdf_name)
        
        # Use the improved function with exact W-9 scaling
        result = create_patient_bill_open_only_exact_w9(merged, fn, ln, dob, output_path=pdf_path, entity=entity_sel)
        
        if not result:
            print('No matching Open case found. Check name, DOB, and Entity.')
            return
        
        print(f"Bill created: {result['pdf_path']}")
        print(f"Output folder: {output_dir}")
        print(f"Entity: {entity_sel or 'Any'}")
        print(f"Open cases only: ✓")
        print(f"W-9 scaling: Exact page dimensions")
        print(f"W-9 detection: Enhanced with debugging")
        
        # Store result in a way that's accessible to the pdf_out section
        global last_result
        last_result = result
    
    with pdf_out:
        if 'last_result' in globals() and last_result:
            rel = last_result['pdf_path']
            display(HTML(f"<embed src='{rel}' type='application/pdf' width='100%' height='800px' />"))

generate_btn.on_click(on_generate_clicked)

form = widgets.VBox([
    widgets.HBox([first_name_w, last_name_w, dob_w]),
    widgets.HBox([entity_w]),
    generate_btn,
    status_out,
    pdf_out
])

display(form)


In [62]:
# # GUI Interface
# # Input widgets
# first_name_w = widgets.Text(description='First Name', placeholder='e.g., Michelle')
# last_name_w = widgets.Text(description='Last Name', placeholder='e.g., Taylor')
# dob_w = widgets.Text(description='DOB', placeholder='MM/DD/YYYY')
# out_folder_w = widgets.Text(description='Save To', placeholder='e.g., invoices_specific', value='invoices_specific')

# # Entity dropdown
# if 'Entity' in merged.columns:
#     entity_values = sorted(list({str(v) for v in merged['Entity'].dropna().unique()}))
#     entity_options = ['Any'] + entity_values
# else:
#     entity_options = ['Any']
# entity_w = widgets.Dropdown(options=entity_options, value='Any', description='Entity')

# generate_btn = widgets.Button(description='Generate Bill with W-9 (Open Cases Only)', button_style='primary', icon='file')
# status_out = widgets.Output()
# pdf_out = widgets.Output()

# def on_generate_clicked(_):
#     status_out.clear_output()
#     pdf_out.clear_output()
    
#     with status_out:
#         fn = first_name_w.value.strip()
#         ln = last_name_w.value.strip()
#         dob_str = dob_w.value.strip()
#         out_dir = out_folder_w.value.strip() or '.'
#         entity_sel = None if entity_w.value == 'Any' else entity_w.value

#         if not fn or not ln or not dob_str:
#             print('Please fill First Name, Last Name, and DOB.')
#             return
        
#         try:
#             dob = pd.to_datetime(dob_str, format='%m/%d/%Y')
#         except Exception:
#             try:
#                 dob = pd.to_datetime(dob_str)
#             except Exception:
#                 print('DOB format invalid. Use MM/DD/YYYY.')
#                 return
        
#         # Create output directory structure: ../Output_Folder/{invoice_name}
#         output_base_dir = "../Output_Folder"
#         invoice_name = out_dir.strip() or 'invoices_specific'
#         output_dir = os.path.join(output_base_dir, invoice_name)
#         os.makedirs(output_dir, exist_ok=True)
        
#         # Get entity name for filename
#         entity_name = entity_sel or 'Any'
#         entity_clean = entity_name.replace(' ', '_').replace(',', '').replace('.', '').replace('&', 'and').replace('LLC', '').replace('INC', '').strip('_')
        
#         pdf_name = f"{fn}_{ln}_Bill_{entity_clean}_{datetime.now().strftime('%Y%m%d')}.pdf".replace(' ', '_')
#         pdf_path = os.path.join(output_dir, pdf_name)
        
#         # Use the improved function with exact W-9 scaling
#         result = create_patient_bill_open_only_exact_w9(merged, fn, ln, dob, output_path=pdf_path, entity=entity_sel)
        
#         if not result:
#             print('No matching Open case found. Check name, DOB, and Entity.')
#             return
        
#         print(f"Bill created: {result['pdf_path']}")
#         print(f"Output folder: {output_dir}")
#         print(f"Entity: {entity_sel or 'Any'}")
#         print(f"Open cases only: ✓")
#         print(f"W-9 scaling: Exact page dimensions")
#         print(f"W-9 detection: Enhanced with debugging")
        
#         # Store result in a way that's accessible to the pdf_out section
#         global last_result
#         last_result = result
    
#     with pdf_out:
#         if 'last_result' in globals() and last_result:
#             rel = last_result['pdf_path']
#             display(HTML(f"<embed src='{rel}' type='application/pdf' width='100%' height='800px' />"))

# generate_btn.on_click(on_generate_clicked)

# form = widgets.VBox([
#     widgets.HBox([first_name_w, last_name_w, dob_w]),
#     widgets.HBox([entity_w, out_folder_w]),
#     generate_btn,
#     status_out,
#     pdf_out
# ])

# display(form)


# Alternative: Non-Widget Version

If the ipywidgets don't work, you can use this alternative approach that uses simple input() functions instead of widgets:

```python
# Alternative GUI without widgets
def generate_bill_simple():
    """Simple version without widgets - uses input() prompts"""
    print("=== Billing System - Simple Version ===")
    
    # Get user input
    first_name = input("Enter First Name: ").strip()
    last_name = input("Enter Last Name: ").strip()
    dob_str = input("Enter DOB (MM/DD/YYYY): ").strip()
    
    # Entity selection
    print("\nAvailable Entities:")
    entities = list(merged['Entity'].dropna().unique())
    for i, entity in enumerate(entities, 1):
        print(f"{i}. {entity}")
    print("0. Any Entity")
    
    try:
        choice = int(input("Select Entity (number): "))
        if choice == 0:
            entity_sel = None
        else:
            entity_sel = entities[choice - 1]
    except (ValueError, IndexError):
        print("Invalid choice, using 'Any Entity'")
        entity_sel = None
    
    if not first_name or not last_name or not dob_str:
        print('Please fill First Name, Last Name, and DOB.')
        return
    
    try:
        dob = pd.to_datetime(dob_str, format='%m/%d/%Y')
    except Exception:
        try:
            dob = pd.to_datetime(dob_str)
        except Exception:
            print('DOB format invalid. Use MM/DD/YYYY.')
            return
    
    # Create output directory
    output_base_dir = "../Output_Folder"
    invoice_name = f"{first_name}_{last_name}_Bills"
    output_dir = os.path.join(output_base_dir, invoice_name)
    os.makedirs(output_dir, exist_ok=True)
    
    # Generate filename
    entity_name = entity_sel or 'Any'
    entity_clean = entity_name.replace(' ', '_').replace(',', '').replace('.', '').replace('&', 'and').replace('LLC', '').replace('INC', '').strip('_')
    pdf_name = f"{first_name}_{last_name}_Bill_{entity_clean}_{datetime.now().strftime('%Y%m%d')}.pdf".replace(' ', '_')
    pdf_path = os.path.join(output_dir, pdf_name)
    
    # Generate the bill
    result = create_patient_bill_open_only_exact_w9(merged, first_name, last_name, dob, output_path=pdf_path, entity=entity_sel)
    
    if not result:
        print('No matching Open case found. Check name, DOB, and Entity.')
        return
    
    print(f"\n✅ Bill created successfully!")
    print(f"📁 Output folder: {output_dir}")
    print(f"📄 PDF file: {pdf_name}")
    print(f"🏢 Entity: {entity_sel or 'Any'}")
    print(f"📋 Open cases only: ✓")
    
    return result

# Uncomment the line below to use the simple version instead of widgets
# generate_bill_simple()
```


In [63]:
# FIXED VERSION: get_patient_bill_data_open_only with deduplication
def get_patient_bill_data_open_only_fixed(merged_df, first_name, last_name, dob, entity=None):
    """Get comprehensive bill data for a specific patient with 'Open' case status only - WITH DEDUPLICATION and TRACING."""
    import pandas as pd
    
    first_name_norm = first_name.strip().lower()
    last_name_norm = last_name.strip().lower()
    
    if isinstance(dob, str):
        try:
            dob_parsed = pd.to_datetime(dob, format='%m/%d/%Y')
        except Exception:
            try:
                dob_parsed = pd.to_datetime(dob)
            except Exception:
                return None, None, None, None
    else:
        dob_parsed = dob
    
    mask = (
        (merged_df['First Name'].str.lower().str.strip() == first_name_norm) &
        (merged_df['Last Name'].str.lower().str.strip() == last_name_norm) &
        (pd.to_datetime(merged_df['patient_dob'], errors='coerce') == dob_parsed) &
        (merged_df['Case Status'] == 'Open')  # Only include 'Open' cases
    )
    
    if entity and str(entity).strip():
        mask &= (merged_df['Entity'].astype(str).str.strip().str.lower() == str(entity).strip().lower())
    
    patient_data = merged_df[mask]
    
    # Save filtered dataframe snapshot for debugging
    if TRACE_ENABLED:
        try:
            _trace_store['filtered_rows'] = patient_data.copy()
        except Exception:
            pass
    
    trace('filter_applied',
          first_name=first_name_norm,
          last_name=last_name_norm,
          dob=str(dob_parsed.date()) if hasattr(dob_parsed, 'date') else str(dob_parsed),
          entity=(str(entity).strip().lower() if entity else 'any'),
          matched_rows=int(patient_data.shape[0]))
    
    if patient_data.empty:
        print(f"No 'Open' case found with name: {first_name} {last_name}, DOB: {dob}, Entity: {entity or 'Any'}")
        return None, None, None, None
    
    # Use the most recent row by Date of Service to reflect latest Case Rep/attorney info
    try:
        patient_data_sorted = patient_data.sort_values(
            by=['Date of Service'],
            key=lambda s: s.apply(_safe_to_datetime)
        ).reset_index(drop=True)
        first_row = patient_data_sorted.iloc[-1]
    except Exception:
        # Fallback: last row in current order
        first_row = patient_data.iloc[-1]
    
    # Get patient address as string, not Series
    patient_address = first_row.get('patient_address', 'Unknown Address')
    if hasattr(patient_address, 'iloc'):
        # If it's a Series, get the first value
        patient_address = str(patient_address.iloc[0]) if len(patient_address) > 0 else 'Unknown Address'
    else:
        patient_address = str(patient_address)
    
    patient_info = {
        'Patient Fname': first_row.get('First Name', 'Unknown'),
        'Patient Lname': first_row.get('Last Name', 'Unknown'),
        'DOB': first_row.get('patient_dob', 'Unknown'),
        'DOI': first_row.get('patient_doi', 'Unknown'),
        'Patient Phone': first_row.get('patient_patient_phone', 'Unknown'),
        'patient_address': patient_address,
        'patient_street': first_row.get('patient_street', ''),
        'patient_city': first_row.get('patient_city', ''),
        'patient_state': first_row.get('patient_state', ''),
        'patient_zip': first_row.get('patient_zip', ''),
        'Entity': first_row.get('Entity', 'Unknown Entity')  # Add Entity information for proper branding
    }
    
    attorney_info = {
        'Attorney Name': first_row.get('attorney_attorney_name', first_row.get('Attorney', 'Unknown Attorney')),
        'Case Rep': first_row.get('Case Rep', 'Unknown'),
        'Firm': first_row.get('attorney_firm', first_row.get('Law Firm', 'Unknown Firm')),
        'Address': first_row.get('attorney_address', 'Unknown Address'),
        'City': first_row.get('attorney_city', 'Unknown City'),
        'State': first_row.get('attorney_state', 'Unknown State'),
        'Zip': first_row.get('attorney_zip', 'Unknown Zip'),
        'attorney_email': first_row.get('attorney_email', ''),
        'case_rep_email': first_row.get('case_rep_email', ''),
        'case_rep_phone': first_row.get('case_rep_phone', ''),
        'case_rep_fax': first_row.get('case_rep_fax', '')
    }
    
    procedures = []
    total_amount = 0.0
    
    # Create a set to track unique procedures and avoid duplicates
    seen_procedures = set()
    duplicate_keys = []
    
    for _, row in patient_data.iterrows():
        charge_amount = 0.0
        if pd.notna(row.get('Charge Amount')):
            charge_amount = float(row['Charge Amount'])
        elif pd.notna(row.get('Item Charge')):
            charge_amount = float(row['Item Charge'])
        
        # Create a unique identifier for this procedure to detect duplicates
        date_of_service = row.get('Date of Service', 'Unknown')
        service_description = row.get('Full Service Description', 'Unknown Service')
        
        # Convert date to string for consistent comparison
        if hasattr(date_of_service, 'strftime'):
            date_str = date_of_service.strftime('%Y-%m-%d')
        else:
            date_str = str(date_of_service)
        
        # Create unique key based on date, description, and charge amount
        procedure_key = f"{date_str}|{service_description}|{charge_amount}"
        
        # Only add if we haven't seen this exact procedure before
        if procedure_key not in seen_procedures:
            procedure = {
                'Date of Service': date_of_service,
                'Full Service Description': service_description,
                'Item Charge': row.get('Item Charge', 0),
                'Charge Amount': row.get('Charge Amount', 0),
                'Final Charge': charge_amount
            }
            
            procedures.append(procedure)
            seen_procedures.add(procedure_key)
            total_amount += charge_amount
        else:
            duplicate_keys.append(procedure_key)
    
    # Sort procedures by Date of Service in chronological order
    def sort_key(procedure):
        date_of_service = procedure['Date of Service']
        if pd.isna(date_of_service) or date_of_service == 'Unknown':
            return pd.Timestamp.min  # Put unknown dates at the end
        try:
            if isinstance(date_of_service, str):
                return pd.to_datetime(date_of_service, errors='coerce')
            return pd.to_datetime(date_of_service, errors='coerce')
        except:
            return pd.Timestamp.min  # Put invalid dates at the end
    
    procedures.sort(key=sort_key)
    
    trace('dedup_summary',
          duplicates_detected=len(duplicate_keys),
          unique_procedures=len(procedures),
          total_amount=float(total_amount))
    
    return patient_info, attorney_info, procedures, total_amount


In [64]:
# FIXED VERSION: create_patient_bill_open_only_exact_w9 with deduplication
def create_patient_bill_open_only_exact_w9_fixed(merged_df, first_name, last_name, dob, output_path=None, entity=None):
    """Complete function to get patient data and generate a PDF bill for 'Open' cases only with exact W-9 scaling - WITH DEDUPLICATION and TRACING."""
    
    patient_info, attorney_info, procedures, total_amount = get_patient_bill_data_open_only_fixed(
        merged_df, first_name, last_name, dob, entity=entity
    )
    
    if patient_info is None:
        return None
    
    if output_path is None:
        safe_name = f"{first_name}_{last_name}_bill.pdf".replace(" ", "_")
        output_path = safe_name
    
    # If tracing enabled, write debug artifacts
    try:
        if TRACE_ENABLED:
            debug_dir = os.path.join(os.path.dirname(output_path), 'Debug')
            os.makedirs(debug_dir, exist_ok=True)
            import json
            with open(os.path.join(debug_dir, 'trace_events.json'), 'w', encoding='utf-8') as f:
                json.dump(_trace_events, f, indent=2, default=str)
            # Also dump filtered rows if present
            df_filtered = _trace_store.get('filtered_rows')
            if df_filtered is not None:
                df_path = os.path.join(debug_dir, 'filtered_rows.csv')
                try:
                    df_filtered.to_csv(df_path, index=False)
                except Exception:
                    pass
    except Exception as _:
        pass
    
    pdf_path = generate_pdf_bill_with_exact_w9_scaling(patient_info, attorney_info, procedures, total_amount, output_path)
    
    return {
        'patient_info': patient_info,
        'attorney_info': attorney_info, 
        'procedures': procedures,
        'total_amount': total_amount,
        'pdf_path': pdf_path,
        'num_procedures': len(procedures)
    }


In [65]:
# Updated GUI Interface with DEDUPLICATION FIX
# Input widgets with proper copy/paste support
first_name_w = widgets.Text(
    description='First Name', 
    placeholder='e.g., Michelle', 
    layout=widgets.Layout(width='200px'),
    style={'description_width': 'initial'}
)
last_name_w = widgets.Text(
    description='Last Name', 
    placeholder='e.g., Taylor', 
    layout=widgets.Layout(width='200px'),
    style={'description_width': 'initial'}
)
dob_w = widgets.Text(
    description='DOB', 
    placeholder='MM/DD/YYYY', 
    layout=widgets.Layout(width='200px'),
    style={'description_width': 'initial'}
)

# Entity dropdown
if 'Entity' in merged.columns:
    entity_values = sorted(list({str(v) for v in merged['Entity'].dropna().unique()}))
    entity_options = ['Any'] + entity_values
else:
    entity_options = ['Any']
entity_w = widgets.Dropdown(options=entity_options, value='Any', description='Entity')

trace_toggle = widgets.Checkbox(value=False, description='Enable Tracing (write Debug folder)', indent=False)

generate_btn = widgets.Button(description='Generate Bill with W-9 (Open Cases Only) - FIXED', button_style='primary', icon='file')
status_out = widgets.Output()
pdf_out = widgets.Output()

def on_generate_clicked(_):
    status_out.clear_output()
    pdf_out.clear_output()
    
    with status_out:
        fn = first_name_w.value.strip()
        ln = last_name_w.value.strip()
        dob_str = dob_w.value.strip()
        entity_sel = None if entity_w.value == 'Any' else entity_w.value

        if not fn or not ln or not dob_str:
            print('Please fill First Name, Last Name, and DOB.')
            return
        
        try:
            dob = pd.to_datetime(dob_str, format='%m/%d/%Y')
        except Exception:
            try:
                dob = pd.to_datetime(dob_str)
            except Exception:
                print('DOB format invalid. Use MM/DD/YYYY.')
                return
        
        # Tracing toggle
        set_trace(trace_toggle.value)
        if TRACE_ENABLED:
            print('Tracing enabled. Debug artifacts will be written next to the PDF file.')
            # reset containers for a clean run
            global _trace_events, _trace_store
            _trace_events = []
            _trace_store = {}
        
        # Create output directory structure: ../Output_Folder/{PatientFirstName}_{PatientLastName}_Bills
        output_base_dir = "../Output_Folder"
        invoice_name = f"{fn}_{ln}_Bills"
        output_dir = os.path.join(output_base_dir, invoice_name)
        os.makedirs(output_dir, exist_ok=True)
        
        # Get entity name for filename
        entity_name = entity_sel or 'Any'
        entity_clean = entity_name.replace(' ', '_').replace(',', '').replace('.', '').replace('&', 'and').replace('LLC', '').replace('INC', '').strip('_')
        
        pdf_name = f"{fn}_{ln}_Bill_{entity_clean}_{datetime.now().strftime('%Y%m%d')}.pdf".replace(' ', '_')
        pdf_path = os.path.join(output_dir, pdf_name)
        
        # Use the FIXED function with deduplication
        result = create_patient_bill_open_only_exact_w9_fixed(merged, fn, ln, dob, output_path=pdf_path, entity=entity_sel)
        
        if not result:
            print('No matching Open case found. Check name, DOB, and Entity.')
            return
        
        print(f"Bill created: {result['pdf_path']}")
        print(f"Output folder: {output_dir}")
        print(f"Entity: {entity_sel or 'Any'}")
        print(f"Open cases only: ✓")
        print(f"W-9 scaling: Exact page dimensions")
        print(f"W-9 detection: Enhanced with debugging")
        print(f"DUPLICATE RECORDS: FIXED - Deduplication applied")
        print(f"Procedures found: {result['num_procedures']} (duplicates removed)")
        if TRACE_ENABLED:
            print("Debug files written in: " + os.path.join(output_dir, 'Debug'))
        
        # Store result in a way that's accessible to the pdf_out section
        global last_result
        last_result = result
    
    with pdf_out:
        if 'last_result' in globals() and last_result:
            rel = last_result['pdf_path']
            display(HTML(f"<embed src='{rel}' type='application/pdf' width='100%' height='800px' />"))

generate_btn.on_click(on_generate_clicked)

form = widgets.VBox([
    widgets.HBox([first_name_w, last_name_w, dob_w]),
    widgets.HBox([entity_w, trace_toggle]),
    generate_btn,
    status_out,
    pdf_out
])

display(form)
